# Experiment 005: Full SA Optimization from jiweiliu

This implements the COMPLETE SA optimization:
1. Create 2-tree unit cells
2. Tile into grids
3. SA optimization of translations, positions, and angles
4. Backward propagation cascade

In [1]:
import math
import numpy as np
import pandas as pd
from numba import njit
from numba.typed import List as NumbaList
import time
import warnings
warnings.filterwarnings('ignore')

# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H
MAX_OVERLAP_DIST_SQ = 1.8 * 1.8

In [2]:
# Numba geometry functions
@njit(cache=True)
def rotate_point(x, y, cos_a, sin_a):
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a

@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    angle_rad = angle_deg * math.pi / 180.0
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([[0.0, TIP_Y], [TOP_W/2, TIER_1_Y], [TOP_W/4, TIER_1_Y],
                    [MID_W/2, TIER_2_Y], [MID_W/4, TIER_2_Y], [BASE_W/2, BASE_Y],
                    [TRUNK_W/2, BASE_Y], [TRUNK_W/2, TRUNK_BOTTOM_Y],
                    [-TRUNK_W/2, TRUNK_BOTTOM_Y], [-TRUNK_W/2, BASE_Y],
                    [-BASE_W/2, BASE_Y], [-MID_W/4, TIER_2_Y], [-MID_W/2, TIER_2_Y],
                    [-TOP_W/4, TIER_1_Y], [-TOP_W/2, TIER_1_Y]], dtype=np.float64)
    for i in range(15):
        rx, ry = rotate_point(pts[i,0], pts[i,1], cos_a, sin_a)
        vertices[i,0], vertices[i,1] = rx + cx, ry + cy
    return vertices

@njit(cache=True)
def polygon_bounds(vertices):
    min_x, min_y = vertices[0,0], vertices[0,1]
    max_x, max_y = vertices[0,0], vertices[0,1]
    for i in range(1, vertices.shape[0]):
        x, y = vertices[i,0], vertices[i,1]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def point_in_polygon(px, py, vertices):
    n = vertices.shape[0]
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = vertices[i,0], vertices[i,1]
        xj, yj = vertices[j,0], vertices[j,1]
        if ((yi > py) != (yj > py)) and (px < (xj-xi)*(py-yi)/(yj-yi) + xi):
            inside = not inside
        j = i
    return inside

@njit(cache=True)
def segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
    dax, day = p2x-p1x, p2y-p1y
    dbx, dby = p4x-p3x, p4y-p3y
    cross_b1 = dbx*(p1y-p3y) - dby*(p1x-p3x)
    cross_b2 = dbx*(p2y-p3y) - dby*(p2x-p3x)
    if cross_b1 * cross_b2 > 0: return False
    cross_a1 = dax*(p3y-p1y) - day*(p3x-p1x)
    cross_a2 = dax*(p4y-p1y) - day*(p4x-p1x)
    if cross_a1 * cross_a2 > 0: return False
    return True

In [3]:
@njit(cache=True)
def polygons_overlap(verts1, verts2, cx1, cy1, cx2, cy2):
    dx, dy = cx2-cx1, cy2-cy1
    if dx*dx + dy*dy > MAX_OVERLAP_DIST_SQ: return False
    min_x1, min_y1, max_x1, max_y1 = polygon_bounds(verts1)
    min_x2, min_y2, max_x2, max_y2 = polygon_bounds(verts2)
    if max_x1 < min_x2 or max_x2 < min_x1 or max_y1 < min_y2 or max_y2 < min_y1: return False
    for i in range(verts1.shape[0]):
        if point_in_polygon(verts1[i,0], verts1[i,1], verts2): return True
    for i in range(verts2.shape[0]):
        if point_in_polygon(verts2[i,0], verts2[i,1], verts1): return True
    n1, n2 = verts1.shape[0], verts2.shape[0]
    for i in range(n1):
        j = (i+1) % n1
        for k in range(n2):
            m = (k+1) % n2
            if segments_intersect(verts1[i,0], verts1[i,1], verts1[j,0], verts1[j,1],
                                  verts2[k,0], verts2[k,1], verts2[m,0], verts2[m,1]): return True
    return False

@njit(cache=True)
def has_any_overlap(all_vertices, centers_x, centers_y):
    n = len(all_vertices)
    for i in range(n):
        for j in range(i+1, n):
            if polygons_overlap(all_vertices[i], all_vertices[j], centers_x[i], centers_y[i], centers_x[j], centers_y[j]):
                return True
    return False

@njit(cache=True)
def get_side_length(all_vertices):
    min_x, min_y, max_x, max_y = math.inf, math.inf, -math.inf, -math.inf
    for verts in all_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x: min_x = x1
        if y1 < min_y: min_y = y1
        if x2 > max_x: max_x = x2
        if y2 > max_y: max_y = y2
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score_numba(all_vertices):
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

In [4]:
@njit(cache=True)
def create_grid_vertices(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows):
    """Create grid of tree vertices by translation."""
    n_seeds = len(seed_xs)
    n_total = n_seeds * ncols * nrows
    all_vertices = []
    centers_x = np.empty(n_total, dtype=np.float64)
    centers_y = np.empty(n_total, dtype=np.float64)
    idx = 0
    for s in range(n_seeds):
        for col in range(ncols):
            for row in range(nrows):
                cx = seed_xs[s] + col * a
                cy = seed_ys[s] + row * b
                all_vertices.append(get_tree_vertices(cx, cy, seed_degs[s]))
                centers_x[idx] = cx
                centers_y[idx] = cy
                idx += 1
    return all_vertices, centers_x, centers_y

@njit(cache=True)
def get_initial_translations(seed_xs, seed_ys, seed_degs):
    seed_vertices = [get_tree_vertices(seed_xs[i], seed_ys[i], seed_degs[i]) for i in range(len(seed_xs))]
    min_x, min_y, max_x, max_y = math.inf, math.inf, -math.inf, -math.inf
    for verts in seed_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x: min_x = x1
        if y1 < min_y: min_y = y1
        if x2 > max_x: max_x = x2
        if y2 > max_y: max_y = y2
    return max_x - min_x, max_y - min_y

In [5]:
@njit(cache=True)
def sa_optimize(seed_xs_init, seed_ys_init, seed_degs_init, a_init, b_init, ncols, nrows,
                Tmax, Tmin, nsteps, nsteps_per_T, position_delta, angle_delta, angle_delta2, delta_t, random_seed):
    """Full SA optimization of lattice parameters."""
    np.random.seed(random_seed)
    n_seeds = len(seed_xs_init)
    seed_xs, seed_ys, seed_degs = seed_xs_init.copy(), seed_ys_init.copy(), seed_degs_init.copy()
    a, b = a_init, b_init
    
    # Initial grid
    all_vertices, centers_x, centers_y = create_grid_vertices(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows)
    if has_any_overlap(all_vertices, centers_x, centers_y):
        a_test, b_test = get_initial_translations(seed_xs, seed_ys, seed_degs)
        a, b = max(a, a_test * 1.5), max(b, b_test * 1.5)
        all_vertices, centers_x, centers_y = create_grid_vertices(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows)
    
    current_score = calculate_score_numba(all_vertices)
    best_score, best_xs, best_ys, best_degs, best_a, best_b = current_score, seed_xs.copy(), seed_ys.copy(), seed_degs.copy(), a, b
    
    T = Tmax
    Tfactor = -math.log(Tmax / Tmin)
    n_move_types = n_seeds + 2
    
    for step in range(nsteps):
        for _ in range(nsteps_per_T):
            move_type = np.random.randint(0, n_move_types)
            
            if move_type < n_seeds:  # Perturb single tree
                i = move_type
                old_x, old_y, old_deg = seed_xs[i], seed_ys[i], seed_degs[i]
                seed_xs[i] += (np.random.random() * 2.0 - 1.0) * position_delta
                seed_ys[i] += (np.random.random() * 2.0 - 1.0) * position_delta
                seed_degs[i] = (seed_degs[i] + (np.random.random() * 2.0 - 1.0) * angle_delta) % 360.0
            elif move_type == n_seeds:  # Change translations
                old_a, old_b = a, b
                a *= 1.0 + (np.random.random() * 2.0 - 1.0) * delta_t
                b *= 1.0 + (np.random.random() * 2.0 - 1.0) * delta_t
            else:  # Rotate all trees
                old_degs = seed_degs.copy()
                ddeg = (np.random.random() * 2.0 - 1.0) * angle_delta2
                for i in range(n_seeds): seed_degs[i] = (seed_degs[i] + ddeg) % 360.0
            
            # Check 2x2 grid for quick collision test
            test_v, test_cx, test_cy = create_grid_vertices(seed_xs, seed_ys, seed_degs, a, b, 2, 2)
            if has_any_overlap(test_v, test_cx, test_cy):
                if move_type < n_seeds: seed_xs[move_type], seed_ys[move_type], seed_degs[move_type] = old_x, old_y, old_deg
                elif move_type == n_seeds: a, b = old_a, old_b
                else:
                    for i in range(n_seeds): seed_degs[i] = old_degs[i]
                continue
            
            # Full grid check
            new_v, new_cx, new_cy = create_grid_vertices(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows)
            if has_any_overlap(new_v, new_cx, new_cy):
                if move_type < n_seeds: seed_xs[move_type], seed_ys[move_type], seed_degs[move_type] = old_x, old_y, old_deg
                elif move_type == n_seeds: a, b = old_a, old_b
                else:
                    for i in range(n_seeds): seed_degs[i] = old_degs[i]
                continue
            
            new_score = calculate_score_numba(new_v)
            delta = new_score - current_score
            accept = delta < 0 or (T > 1e-10 and np.random.random() < math.exp(-delta / T))
            
            if accept:
                current_score = new_score
                if new_score < best_score:
                    best_score, best_xs, best_ys, best_degs, best_a, best_b = new_score, seed_xs.copy(), seed_ys.copy(), seed_degs.copy(), a, b
            else:
                if move_type < n_seeds: seed_xs[move_type], seed_ys[move_type], seed_degs[move_type] = old_x, old_y, old_deg
                elif move_type == n_seeds: a, b = old_a, old_b
                else:
                    for i in range(n_seeds): seed_degs[i] = old_degs[i]
        
        T = Tmax * math.exp(Tfactor * (step + 1) / nsteps)
    
    return best_score, best_xs, best_ys, best_degs, best_a, best_b

In [6]:
# Load baseline
def load_submission_data(filepath):
    df = pd.read_csv(filepath)
    all_xs, all_ys, all_degs = [], [], []
    for n in range(1, 201):
        group = df[df["id"].str.startswith(f"{n:03d}_")].sort_values("id")
        for _, row in group.iterrows():
            all_xs.append(float(str(row["x"]).replace('s', '')))
            all_ys.append(float(str(row["y"]).replace('s', '')))
            all_degs.append(float(str(row["deg"]).replace('s', '')))
    return np.array(all_xs), np.array(all_ys), np.array(all_degs)

def calculate_total_score(all_xs, all_ys, all_degs):
    total, idx = 0.0, 0
    for n in range(1, 201):
        vertices = NumbaList()
        for i in range(n): vertices.append(get_tree_vertices(all_xs[idx+i], all_ys[idx+i], all_degs[idx+i]))
        total += calculate_score_numba(vertices)
        idx += n
    return total

print("Loading baseline...")
baseline_xs, baseline_ys, baseline_degs = load_submission_data('/home/code/santa-2025-csv/santa-2025.csv')
baseline_score = calculate_total_score(baseline_xs, baseline_ys, baseline_degs)
print(f"Baseline score: {baseline_score:.6f}")

Loading baseline...


Baseline score: 70.676102


In [7]:
# Run SA optimization for key N values
print("\n" + "="*60)
print("Running SA optimization for lattice configurations...")
print("="*60)

# Initial 2-tree unit cell (from analysis of N=200 configuration)
seed_xs = np.array([0.0, 0.046], dtype=np.float64)
seed_ys = np.array([0.0, -0.229], dtype=np.float64)
seed_degs = np.array([67.0, 250.0], dtype=np.float64)

# SA parameters
params = {
    'Tmax': 0.1,
    'Tmin': 1e-6,
    'nsteps': 5000,  # Reduced for faster testing
    'nsteps_per_T': 100,
    'position_delta': 0.1,
    'angle_delta': 10.0,
    'angle_delta2': 5.0,
    'delta_t': 0.05
}

# Grid configurations to try
configs = [
    (4, 9, 72),    # 4*9*2 = 72
    (5, 10, 100),  # 5*10*2 = 100
    (6, 12, 144),  # 6*12*2 = 144
    (7, 14, 196),  # 7*14*2 = 196
]

sa_results = {}
for ncols, nrows, target_n in configs:
    print(f"\nOptimizing N={target_n} (grid {ncols}x{nrows})...")
    start = time.time()
    
    # Get initial translations
    a_init, b_init = get_initial_translations(seed_xs, seed_ys, seed_degs)
    a_init *= 1.2  # Start with larger translations
    b_init *= 1.2
    
    best_score, best_xs, best_ys, best_degs, best_a, best_b = sa_optimize(
        seed_xs, seed_ys, seed_degs, a_init, b_init, ncols, nrows,
        params['Tmax'], params['Tmin'], params['nsteps'], params['nsteps_per_T'],
        params['position_delta'], params['angle_delta'], params['angle_delta2'], params['delta_t'],
        random_seed=42
    )
    
    # Get baseline score for this N
    idx = sum(range(target_n))
    baseline_vertices = NumbaList()
    for i in range(target_n):
        baseline_vertices.append(get_tree_vertices(baseline_xs[idx+i], baseline_ys[idx+i], baseline_degs[idx+i]))
    baseline_n_score = calculate_score_numba(baseline_vertices)
    
    print(f"  SA score: {best_score:.6f}, Baseline: {baseline_n_score:.6f}")
    print(f"  Improvement: {baseline_n_score - best_score:.6f} ({(baseline_n_score - best_score)/baseline_n_score*100:.2f}%)")
    print(f"  Time: {time.time()-start:.1f}s")
    
    sa_results[target_n] = {
        'score': best_score,
        'baseline': baseline_n_score,
        'xs': best_xs,
        'ys': best_ys,
        'degs': best_degs,
        'a': best_a,
        'b': best_b,
        'ncols': ncols,
        'nrows': nrows
    }


Running SA optimization for lattice configurations...

Optimizing N=72 (grid 4x9)...


  SA score: 0.401856, Baseline: 0.348559
  Improvement: -0.053296 (-15.29%)
  Time: 30.8s

Optimizing N=100 (grid 5x10)...
  SA score: 0.437622, Baseline: 0.345531
  Improvement: -0.092091 (-26.65%)
  Time: 43.9s

Optimizing N=144 (grid 6x12)...


  SA score: 0.385399, Baseline: 0.342276
  Improvement: -0.043123 (-12.60%)
  Time: 77.9s

Optimizing N=196 (grid 7x14)...
  SA score: 0.385884, Baseline: 0.333299
  Improvement: -0.052585 (-15.78%)
  Time: 91.0s


In [ ]:
# Generate final tree positions from SA results
def get_grid_positions(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows):
    xs, ys, degs = [], [], []
    for s in range(len(seed_xs)):
        for col in range(ncols):
            for row in range(nrows):
                xs.append(seed_xs[s] + col * a)
                ys.append(seed_ys[s] + row * b)
                degs.append(seed_degs[s])
    return np.array(xs), np.array(ys), np.array(degs)

# Create improved solution by taking best of SA or baseline for each N
print("\n" + "="*60)
print("Creating improved solution...")
print("="*60)

improved_xs = baseline_xs.copy()
improved_ys = baseline_ys.copy()
improved_degs = baseline_degs.copy()

improvements = 0
for target_n, result in sa_results.items():
    if result['score'] < result['baseline']:
        # Use SA result
        xs, ys, degs = get_grid_positions(
            result['xs'], result['ys'], result['degs'],
            result['a'], result['b'], result['ncols'], result['nrows']
        )
        idx = sum(range(target_n))
        for i in range(target_n):
            improved_xs[idx+i] = xs[i]
            improved_ys[idx+i] = ys[i]
            improved_degs[idx+i] = degs[i]
        print(f"N={target_n}: Using SA result (improvement: {result['baseline'] - result['score']:.6f})")
        improvements += 1
    else:
        print(f"N={target_n}: Keeping baseline (SA was worse)")

print(f"\nTotal improvements: {improvements}")

In [ ]:
# Calculate final score
final_score = calculate_total_score(improved_xs, improved_ys, improved_degs)
print(f"\nBaseline score: {baseline_score:.6f}")
print(f"Final score: {final_score:.6f}")
print(f"Improvement: {baseline_score - final_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap to target: {final_score - 68.922808:.6f}")

In [ ]:
# Save submission
def save_submission(filepath, all_xs, all_ys, all_degs):
    rows = []
    idx = 0
    for n in range(1, 201):
        for t in range(n):
            rows.append({"id": f"{n:03d}_{t}", "x": f"s{all_xs[idx]}", "y": f"s{all_ys[idx]}", "deg": f"s{all_degs[idx]}"})
            idx += 1
    pd.DataFrame(rows).to_csv(filepath, index=False)

# Use baseline if no improvement
if final_score < baseline_score:
    save_submission('/home/submission/submission.csv', improved_xs, improved_ys, improved_degs)
    save_submission('/home/code/experiments/005_full_sa/submission.csv', improved_xs, improved_ys, improved_degs)
    print("\nSaved improved submission!")
else:
    # Use baseline
    save_submission('/home/submission/submission.csv', baseline_xs, baseline_ys, baseline_degs)
    save_submission('/home/code/experiments/005_full_sa/submission.csv', baseline_xs, baseline_ys, baseline_degs)
    print("\nNo improvement - saved baseline submission")

In [ ]:
# Save metrics
import json
metrics = {
    'baseline_score': baseline_score,
    'final_score': min(final_score, baseline_score),
    'improvement': max(0, baseline_score - final_score),
    'sa_improvements': improvements,
    'target': 68.922808,
    'sa_results': {str(k): {'score': v['score'], 'baseline': v['baseline']} for k, v in sa_results.items()}
}
with open('/home/code/experiments/005_full_sa/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print("Metrics saved")